<a href="https://colab.research.google.com/github/whendoves/huggingface/blob/main/IMDB_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 10.1 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 83.1 MB/s eta 0:00:00


To load IMDB datasets 

In [ ]:
from datasets import load_dataset

imdb = load_dataset('imdb')

imdb

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

Import AutoTokenizer and set checkpoint as "distilbert-base-uncased-finetuned-sst-2-english"

In [ ]:

from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)



Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Depending on fine tuing such as sentiment classification, MASK, NER etc, Tokenizing imdb['text'] is required in IMDB datasets 

In [ ]:
def tokenizing_fn(example):
    
    return tokenizer(example['text'], truncation=True, padding=True)

tokenized_imdb = imdb.map(tokenizing_fn, batched=True)



  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

Load sequence classification model and set num_labels as 2

In [ ]:
from transformers import DataCollatorWithPadding, TFAutoModelForSequenceClassification, create_optimizer

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors='tf')

model = TFAutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english", num_labels=2)




Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


Load tensorflow and keras 

In [ ]:
import tensorflow as tf
from tensorflow import keras

Get column names to be trained, instead of specifying individual coulmn name when loading train datasets

In [ ]:
tokenizer_columns = tokenizer.model_input_names
tokenizer_columns

['input_ids', 'attention_mask']

set batch size, number of epochs and learning rate decay

In [ ]:
batch_size=8
num_epochs = 2
num_train_step = len(tokenized_imdb['train'])*num_epochs
lr_schedule = keras.optimizers.schedules.PolynomialDecay(initial_learning_rate=5e-5, end_learning_rate=0.0,decay_steps=num_train_step)

To pad the entire datasets

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors='tf')

Load train datasets

In [ ]:
tf_train_dataset = tokenized_imdb["train"].to_tf_dataset(columns=tokenizer_columns,
                                                         label_cols=['label'],
                                                         shuffle=True,
                                                          batch_size=batch_size,
                                                          collate_fn = data_collator,
                                                         )

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Load validation datasets

In [ ]:
tf_validation_dataset = tokenized_imdb["test"].to_tf_dataset(columns=tokenizer_columns,
                                                         label_cols=['label'],
                                                         shuffle=False,
                                                          batch_size=batch_size,
                                                          collate_fn = data_collator,
                                                         )

Optimizer

In [ ]:
adam = keras.optimizers.Adam(learning_rate=lr_schedule)

In [ ]:
model.compile(optimizer=adam, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [ ]:
history = model.fit(x=tf_train_dataset, validation_data=tf_validation_dataset, epochs=2)

Epoch 1/2
3125/3125 [==============================] - 1920s 611ms/step - loss: 0.2538 - accuracy: 0.8978 - val_loss: 0.2032 - val_accuracy: 0.9252
Epoch 2/2
3125/3125 [==============================] - 1910s 611ms/step - loss: 0.1380 - accuracy: 0.9504 - val_loss: 0.2580 - val_accuracy: 0.9159
